In [1]:
import openai
import os

from dotenv import load_dotenv
load_dotenv()
openai.api_key  = os.getenv('OPENAI_API_KEY')  # this is not needed for langchain?

In [2]:
# prompting
def get_completion_and_token_count(
    messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500
):
    # messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
        "prompt_tokens": response["usage"]["prompt_tokens"],
        "completion_tokens": response["usage"]["completion_tokens"],
        "total_tokens": response["usage"]["total_tokens"],
    }

    return content, token_dict


messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss.""",
    },
    {
        "role": "user",
        "content": """write me a 4 sentence long poem about a happy carrot""",
    },
]
response, token_dict = get_completion_and_token_count(messages)

print(response)
print(token_dict)

In a garden so bright, a carrot grew,
With a smile so wide, and a vibrant hue.
It danced in the breeze, so full of glee,
A happy carrot, as happy as can be!

With roots in the earth, it reached for the sky,
Its leafy greens waving, saying "hi!"
It shared its joy with all who passed by,
A happy carrot, bringing smiles to the eye!

So let us remember, in life's daily grind,
To be like the carrot, with happiness entwined.
Spread joy and laughter, wherever we go,
Just like the happy carrot, all in a row!
{'prompt_tokens': 37, 'completion_tokens': 128, 'total_tokens': 165}


In [3]:
# moderation
response = openai.Moderation.create(
    input="""
Here's the plan.  We get the warhead, 
and we hold the world ransom...
...FOR ONE MILLION DOLLARS!
"""
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "flagged": false,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": false
  },
  "category_scores": {
    "sexual": 8.910085e-06,
    "hate": 0.00013615482,
    "harassment": 0.0023860661,
    "self-harm": 7.5545418e-06,
    "sexual/minors": 2.20487e-07,
    "hate/threatening": 7.746158e-06,
    "violence/graphic": 0.00012008196,
    "self-harm/intent": 5.9765495e-07,
    "self-harm/instructions": 3.4945369e-09,
    "harassment/threatening": 0.0015225811,
    "violence": 0.34292138
  }
}


In [ ]:
# prompting with LangChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
chat = ChatOpenAI(temperature=0.0)
prompt_template = ChatPromptTemplate.from_template(template_string)
customer_messages = prompt_template.format_messages(
    style=customer_style, text=customer_email
)
customer_response = chat(customer_messages)

In [ ]:
print(prompt_template.messages[0].prompt, end="\n__2__\n")
print(prompt_template.messages[0].prompt.input_variables, end="\n__3__\n")
print(type(customer_messages), end="\n__4__\n")
print(type(customer_messages[0]), end="\n__5__\n")
print(customer_messages[0], end="\n__6__\n")
print(customer_messages[0].content, end="\n__7__\n")
print(customer_response.content)